# Laboratory 2 - Models in credit and operational risk
**Bartłomiej Jamiołkowski 11.03.2022**

## Exercise 1

**What are the differences in applying the standard method under Basel I and Basel II?**

The book 'Risk Management and Financial Institutions' by John C. Hull indicates that the difference between using the standard method according to Basel I and Basel II is the way of calculating risk weights. The new rules have a more specific weighting and the status of some institutions is not taken into account in the same way as in the past.

**How the IRB method works (Vasicek model)**

Based on the book 'Risk Management and Financial Institutions' by John C. Hull, the IRB method can be described in the following steps. The required capital is calculated as the difference between the annual 99.9% VaR and the expected loss. It is written as the sum of the products EADi * LGDi * (WCDRi − PDi) * MA, where EAD is the risk of default, LGD is the loss in the event of default, WCDR is the interest on outstanding loans, and PD is the probability of default.
The correlations for determining some of these quantities can either be read from the table or calculated using formulas.

## Exercise 2

In the capital calculation task, weighting tables 15.1 and 15.4 from the book 'Risk Management and Financial Institutions' by John C. Hull are used.

**Basel I**

Risk-weighted capital.

In [1]:
options("scipen"= 100, "digits"= 4) 
risk_weighted_assets_1 = (200*0.0 + 100*0.2 + 100*1.0)* 1000000
print(paste0(risk_weighted_assets_1, "$"))

[1] "120000000$"


Allowance for potential losses.

In [2]:
write_off_1 = 0.08* risk_weighted_assets_1 
print(paste0(write_off_1, "$"))

[1] "9600000$"


**Basel II**

Capital calculated using the standard method.

In [3]:
risk_weighted_assets_2 = (200*(0.4*0.0 + 0.4*0.2 + 0.2*0.5) + 100*(0.5*0.2 + 0.3*0.5 + 0.1*1.0 + 0.1*0.5) + 
100*(0.2*0.2 + 0.3*0.5 + 0.2*1.0))* 1000000
print(paste0(risk_weighted_assets_2, "$"))

[1] "115000000$"


Allowance for potential losses.

In [4]:
write_off_2 = 0.08* risk_weighted_assets_2 
print(paste0(write_off_2, "$"))

[1] "9200000$"


## Exercise 3

Individual mortgage loans.

In [5]:
PD = 0.005 
correlation = 0.15 # korelacja dla kredytu hipotecznego (prezentacja)
WCDR = 0.063 # tabela w prezentacji
M = 1
MA = 1 
b = (0.11852 − 0.054781*log(PD))^2
LGD = 0.3 # strata na wypadek niewypłacalności
RWA1 = 12.5*20*LGD*(WCDR - PD)
print(paste0(RWA1, " mln $"))

[1] "4.35 mln $"


Loans for companies.

In [6]:
PD = 0.0075 # prawdopodobieństwo niewypłacalności
M = 2 # zapadalność 2-letnia
LGD = 0.6 # strata na wypadek niewypłacalności
N = 3
correlation = 0.12*(1 + exp(-50*PD)) # związek miedzy PD a korelacją wedlug nowej formuły
WCDR = N*((PD/N + sqrt(correlation)/N*0.999)/sqrt(1 - correlation)) # odsetek niespłaconych kredytów w ciągu roku
b = (0.11852 − 0.054781*log(PD))^2
MA = (1 + b*(M - 2.5))/(1 - 1.5*b) # korekta
RWA2 = 12.5*100*LGD*(WCDR - PD)*MA
print(paste0(RWA2, " mln $"))

[1] "451.030784142568 mln $"


Loans for companies.

In [7]:
PD = 0.025 # prawdopodobieństwo niewypłacalności
M = 3 # zapadalność 3-letnia
LGD = 0.6 # strata na wypadek niewypłacalności
N = 3 
correlation = 0.12*(1 + exp(-50*PD)) # związek miedzy PD a korelacją wedlug nowej formuły
WCDR = N*((PD/N + sqrt(correlation)/N*0.999)/sqrt(1 - correlation)) # odsetek niespłaconych kredytów w ciągu roku
b = (0.11852 − 0.054781*log(PD))^2
MA = (1 + b*(M - 2.5))/(1 - 1.5*b) # korekta
RWA3 = 12.5*50*LGD*(WCDR - PD)*MA
print(paste0(RWA3, " mln $"))

[1] "199.990723987108 mln $"


Total RWA for a bank that has three baskets of loans.

In [8]:
RWA = RWA1 + RWA2 + RWA3
print(paste0(RWA, " mln $"))

[1] "655.371508129676 mln $"


## Exercise 4

Libraries used.

In [18]:
options(warn=-1)
library(readxl)
library(dplyr)
library(extraoperators)

Data import.

In [10]:
credits_data <- read_excel("vasicek.xlsx", sheet = "Arkusz1")

Categorization of the PD variable.

In [11]:
credits_data[, "groups"] <- cut(credits_data$PD, breaks = 5, labels = c("Group1", "Group2", "Group3", "Group4", "Group5"))

EAD summation by assigned category.

In [12]:
EAD_list = aggregate(credits_data$EAD, list(credits_data$groups), FUN=sum)  
EAD_list = EAD_list$x
EAD_list

[1] 1725000 1300000  815000  490000  665000

Creating a column with weights and then calculating the weighted average LGD for each category.

In [13]:
weights <- c(3, 4, 5, 6)
credits_data[, "weights"] <- NA
for(i in 1:length(credits_data$LGD)){
    if(credits_data$LGD[i] %gele% c(0.30, 0.39)){
        credits_data$weights[i] = 1
    }
    else if(credits_data$LGD[i] %gele% c(0.40, 0.49)){
        credits_data$weights[i] = 2
    }
    else if(credits_data$LGD[i] %gele% c(0.50, 0.59)){
        credits_data$weights[i] = 3
    }
    else if(credits_data$LGD[i] %gele% c(0.60, 0.69)){
        credits_data$weights[i] = 4
    }
}
weighted_average_LGD <- credits_data %>%                                           # Weighted mean by group
                           group_by(groups) %>% 
                           summarise(weighted.mean(LGD, weights))
colnames(weighted_average_LGD) <- c("PD", "WA")
weighted_average_LGD <- weighted_average_LGD$WA

In [14]:
weighted_average_LGD

[1] 0.5020 0.5000 0.5413 0.4950 0.4522

PD sum by category.

In [15]:
PD_list = aggregate(credits_data$PD, list(credits_data$groups), FUN=sum)
PD_list = PD_list$x
PD_list

[1] 0.0816 0.1987 0.1882 0.1238 0.2143

Calculation of WCDR and RWA for individual categories.

In [16]:
WCDR_list <- c(NA, NA, NA, NA, NA)
RWA_list <- c(NA, NA, NA, NA, NA)
for(i in 1:5){
    correlation = 0.12*(1 + exp(-50*PD_list[i]))
    WCDR_list[i] = 5*((PD_list[i]/5 + sqrt(correlation)/5*0.999)/sqrt(1 - correlation)) 
    RWA_list[i] = 12.5*weighted_average_LGD[1]*EAD_list[i]*(WCDR_list[i] - PD_list[i])
}
RWA_list

[1] 4091193 3116710 1950431 1160900 1598573

Calculation of the RWA link.

In [17]:
RWA = sum(RWA_list)
print(paste0(round(RWA, digits = 2), "$"))

[1] "11917806.83$"
